In [ ]:
import pandas as pd

file_path = '/content/internet_service_churn.csv'
data = pd.read_csv(file_path)

data.info(), data.head()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 72274 entries, 0 to 72273
Data columns (total 11 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   id                           72274 non-null  int64  
 1   is_tv_subscriber             72274 non-null  int64  
 2   is_movie_package_subscriber  72274 non-null  int64  
 3   subscription_age             72274 non-null  float64
 4   bill_avg                     72274 non-null  int64  
 5   reamining_contract           50702 non-null  float64
 6   service_failure_count        72274 non-null  int64  
 7   download_avg                 71893 non-null  float64
 8   upload_avg                   71893 non-null  float64
 9   download_over_limit          72274 non-null  int64  
 10  churn                        72274 non-null  int64  
dtypes: float64(4), int64(7)
memory usage: 6.1 MB


(None,
    id  is_tv_subscriber  is_movie_package_subscriber  subscription_age  \
 0  15                 1                            0             11.95   
 1  18                 0                            0              8.22   
 2  23                 1                            0              8.91   
 3  27                 0                            0              6.87   
 4  34                 0                            0              6.39   
 
    bill_avg  reamining_contract  service_failure_count  download_avg  \
 0        25                0.14                      0           8.4   
 1         0                 NaN                      0           0.0   
 2        16                0.00                      0          13.7   
 3        21                 NaN                      1           0.0   
 4         0                 NaN                      0           0.0   
 
    upload_avg  download_over_limit  churn  
 0         2.3                    0      0  
 1         

***Обробка пропусків***

In [ ]:
#Аналіз відсутніх значень у наборі даних
missing_data = data.isnull().sum()

#Обчислення відсотка відсутніх значень для кожного стовпчика
missing_percentage = (missing_data / len(data)) * 100

#Відображення стовпців з відсутніми значеннями та їх відсотки
missing_info = pd.DataFrame({"Missing Values": missing_data, "Percentage": missing_percentage})
missing_info[missing_info["Missing Values"] > 0]

,Missing Values,Percentage
reamining_contract,21572,29.847525
download_avg,381,0.527161
upload_avg,381,0.527161


In [ ]:
#Заповнюємо пропуски у reamining_contract
data['reamining_contract'].fillna(data['reamining_contract'].median(), inplace=True)

#Заповнити відсутні значення для 'download_avg' та 'upload_avg' відповідними медіанами
data['download_avg'].fillna(data['download_avg'].median(), inplace=True)
data['upload_avg'].fillna(data['upload_avg'].median(), inplace=True)

#Перевірка, що відсутніх значень більше немає
data.isnull().sum()

"""
Було обрано заповнення медіанами, тому що можна зберегти дані без значного внеску.
Якщо б я їх видалив то зламалася б логіка, бо це б означало, що клієнт не має контратку або дані не було зібрано
"""

<ipython-input-4-5b41bb3ea2c8>:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data['reamining_contract'].fillna(data['reamining_contract'].median(), inplace=True)
<ipython-input-4-5b41bb3ea2c8>:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].meth

'\nБуло обрано заповнення медіанами, тому що можна зберегти дані без значного внеску.\nЯкщо б я їх видалив то зламалася б логіка, бо це б означало, що клієнт не має контратку або дані не було зібрано\n'

***Закодувати категоріальні змінні***

In [ ]:
"""
У датасеті всі потенційно категоріальні змінні is_tv_subscriber, is_movie_package_subscriber, download_over_limit, churn вже представлені як числові бінарні змінні 0 або 1.
Це означає, що вони не потребують додаткового кодування.
"""

***Провести нормалізацію/стандартизацію числових ознак***

In [ ]:
"""
У нашому випадку стандартизація буде більш доречною для забезпечення збалансованості вкладу різних ознак
"""

In [ ]:
from sklearn.preprocessing import StandardScaler

#Визначення числових змінних
numerical_columns = ['subscription_age', 'bill_avg', 'reamining_contract', 'download_avg', 'upload_avg']

#Ініціалізація StandardScaler
scaler = StandardScaler()

#Стандартизація
data[numerical_columns] = scaler.fit_transform(data[numerical_columns])

#Вивід результату
print(data.head())

   id  is_tv_subscriber  is_movie_package_subscriber  subscription_age  \
0  15                 1                            0          4.668335   
1  18                 0                            0          2.835389   
2  23                 1                            0          3.174460   
3  27                 0                            0          2.171991   
4  34                 0                            0          1.936116   

   bill_avg  reamining_contract  service_failure_count  download_avg  \
0  0.458372           -0.906023                      0     -0.556631   
1 -1.433376           -0.174330                      0     -0.689440   
2 -0.222657           -1.144248                      0     -0.472835   
3  0.155692           -0.174330                      1     -0.689440   
4 -1.433376           -0.174330                      0     -0.689440   

   upload_avg  download_over_limit  churn  
0   -0.192060                    0      0  
1   -0.426895                    0

In [ ]:
"""
Перевірка правильності
"""

print(data[numerical_columns].mean()) #Середнє значення
print(data[numerical_columns].std()) #Стандартне відхилення

subscription_age      0.000000e+00
bill_avg              7.864989e-18
reamining_contract    0.000000e+00
download_avg          1.101098e-17
upload_avg            3.145995e-17
dtype: float64
subscription_age      1.000007
bill_avg              1.000007
reamining_contract    1.000007
download_avg          1.000007
upload_avg            1.000007
dtype: float64


In [ ]:
data.to_csv('processed_data.csv', index=False)

***Збереження репорту про зроблену працю***

In [ ]:
# Вкажіть текст звіту в змінній report_text
report_text = f'''Обробка відсутніх значень:
reamining_contract: відсутні значення заповнені медіанним значенням.
download_avg: відсутні значення заповнені медіанним значенням.
upload_avg: відсутні значення заповнені медіанним значенням.

Кодування категоріальних змінних:
Змінні is_tv_subscriber, is_movie_package_subscriber, download_over_limit та churn вже мали бінарний формат (0/1). Додаткове кодування не виконувалося.

Стандартизація числових ознак:
На наступні стовпці було застосовано StandardScaler, щоб забезпечити середнє значення = 0 і стандартне відхилення = 1:
subscription_age
bill_avg
reamining_contract
download_avg
upload_avg

Перевірка:
Середнє значення числових змінних після стандартизації: ~0.
Стандартне відхилення числових змінних після стандартизації: ~1.
Оброблені дані збережені у файл: processed_data.csv'''

# Збереження тексту у файл report_table_work.txt
with open('report_table_work.txt', 'w') as f:
    f.write(report_text)

print("Файл збережено як report_table_work.txt")


Файл збережено як report_table_work.txt
